In [1]:
import numpy as np
from aocd import get_data, submit
from operator import itemgetter
import re
import collections

In [2]:
example_data = "        ...#\n        .#..\n        #...\n        ....\n...#.......#\n........#...\n..#....#....\n..........#.\n        ...#....\n        .....#..\n        .#......\n        ......#.\n\n10R5L5R10L4R5L5"
real_data = get_data(day=22, year=2022)

In [3]:
class Board():
    def __init__(self, grid):
        self.grid = grid

    def move(self, current_state, number_of_tiles, cube = False):
        for i in range(number_of_tiles):
            row, column, facing = current_state

            row = row + (+1 if facing == 1 else -1 if facing == 3 else 0)
            column = column + (+1 if facing == 0 else -1 if facing == 2 else 0)

            if row >= grid.shape[0] or column >= grid.shape[1] or grid[row, column] == " ":
                if cube:
                    row, column, facing  = self.cube_wrapping(row, column, facing)
                else:
                    if facing == 0:
                        column = np.argwhere(grid[row] != " ")[0][0]
                    elif facing == 2:
                        column = np.argwhere(grid[row] != " ")[-1][0]
                    elif facing == 1:
                        row = np.argwhere(grid[:,column] != " ")[0][0]
                    else:
                        row = np.argwhere(grid[:,column] != " ")[-1][0]


            if grid[row, column] == "#":
                return current_state

            current_state = (row, column, facing)

        return current_state

    def get_first_state(self):
        column = np.argwhere(grid[0] == ".")[0][0]
        row = 0
        facing = 0
        return (row, column, facing)

    def cube_wrapping(self, row, column, facing):
        # A -> E
        if 150 <= row <200 and column == -1 and facing == 2:
            return 0, row - 100, 1

        # E -> A
        if row == -1 and 50 <= column <100 and facing == 3:
            return column + 100, 0, 0

        #A -> F
        elif row == 200 and 0 <= column < 50 and facing == 1:
            return 0, column + 100, 1

        #F -> A
        elif row == -1 and 100 <= column < 150 and facing == 3:
            return 199, column - 100, 3

        #A -> C
        elif 150 <= row < 200  and column == 50 and facing == 0:
            return 149, row - 100, 3

        #C -> A
        elif row == 150  and 50 <= column < 100 and facing == 1:
            return column + 100, 49, 2

        #B -> E
        elif 100 <= row < 150  and column == -1 and facing == 2:
            return 149-row, 50, 0

        #E -> B
        elif 0 <= row < 50  and column == 49 and facing == 2:
            return 149-row, 0, 0

        #B -> D
        elif row == 99  and 0 <= column < 50 and facing == 3:
            return column+50, 50, 0

        #D -> B
        elif 50 <= row < 100  and column == 49 and facing == 2:
            return 100, row - 50, 1

        #C -> F
        elif 100 <= row < 150  and column == 100 and facing == 0:
            return 149 - row, 149, 2

        #F -> C
        elif 0 <= row < 50  and column == 150 and facing == 0:
            return 149 - row, 99, 2

        #D -> F
        elif 50 <= row < 100  and column == 100 and facing == 0:
            return 49, row + 50, 3

        #F -> D
        elif row == 50  and 100 <= column < 150  and facing == 1:
            return column - 50, 99, 2
        return row, column, facing


In [4]:
raw_map,raw_instructions = real_data.split("\n\n")
instructions = re.findall(r"(\d+|R|L)",raw_instructions)
no_rows = len(raw_map.splitlines())
no_columns = max(map(len,raw_map.splitlines()))
grid = np.array([np.pad(list(line), (0,no_columns-len(line)), constant_values=" ") for line in raw_map.splitlines()])

board = Board(grid)

In [5]:
def find_password(instructions, board, cube=False):
    state = board.get_first_state()
    for instruction in instructions:
        row, column, facing = state
        if instruction == "L":
            state = (row, column, (facing-1)%4)
        elif instruction == "R":
            state = (row, column, (facing+1)%4)
        else:
            state = board.move(state, int(instruction), cube)

    final_row, final_column, final_facing = state
    return 1000 * (final_row+1) + 4 * (final_column+1) + final_facing
print(f"Final password for flat map: : {find_password(instructions, board)}")
print(f"Final password for cube map: : {find_password(instructions, board, True)}")

Final password for flat map: : 3590
Final password for cube map: : 86382
